A working prototype solution to gauge the appropriate unit wise price range for the 3 categories.

# Algorithm/Model used

1. Kmeans for clustering: For extracting patterns in the price column. We can also consider this step as to identify two different patterns: one being acceptable range of values for Price and the other acting as an outlier.
2. Standard deviation method for detcting outliers: Removing values that fall ounder 3 Standard deviations on either side.
 
# Code Applied

We have implemented the code in python. Below is the actual code. 
The program flow is as follows:

1. Importing necessary packages for developing solutions. Packages like numpy,pandas,sklearn,matplotlib has been used.

2. DATA PREPARATION

    1. Data Access: The data was provided in an excel sheet with data residing in various sheets. Here we have appended data from all sheets into one source
    
    2. Data Cleaning: The Unit column has redundant and uncleaned values. You can see below that initially number of distinct categories in Unit column was 97. We have cleaned the data using regex to reduce the distinct categories to 31.
    
3. Data Subsetting and clustering : So the idea is to drill down to category to unit level. For each subset of a given unit and category, we try to create clusters on the Price column. The number of set clusters are two. We select the cluster in which most of the data points falls into. In a sense that the selected cluster predominantly generalizes the data.

4. Outlier Detection: After performing cluster, we select the predominant cluster and perform basic outlier detection to remove data points that fall under extreme ends of data trend. So any value that are above 3 standard deviation on the upper scale and below 3 standard deviation on the lower scale are flaged as outlier and they are removed.

5. Now the filtered and most suitable data is extracted and we create ranges for each category and unit as output.


# Language/Tech used

Python Language.

# Any assumptions taken

1. The actual data will have most intutitive information. For example Price of a product is 5000 and for the same unit is 1000. So the underline assumption is the price is recorded for the same number of quantities. i.e. the number of unit purchased is x and the price is 5000 and for same number of unit x the price is 1000. 
    Per unit price would be more effective. 
2. Seaonality in the data is constant. Say the price for a product is same for all season. Even in the sale perios the price is same.


In [1]:
##Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

DATA ACCESS

In [2]:
sheets_dict = pd.read_excel('data.xlsx', sheetname=None)
data = pd.DataFrame()
for name, sheet in sheets_dict.items():
    sheet['sheet'] = name
    sheet = sheet.rename(columns=lambda x: x.split('\n')[-1])
    data = data.append(sheet)

data.reset_index(inplace=True, drop=True)


In [3]:
data=data.drop('sheet',axis=1)

In [4]:
data['Unit'].unique()

array(['Pair', 'Piece', 'pack', 'Unit', 'Pack', 'Unit/Onwards', 'Pair(s)',
       'unit', 'Set', 'Pieces', 'dollar', 'Packet', 'piece', 'Meter',
       'Piece(s)', 'Per piece', 1, '1pc', 'Box', '170 per peice',
       'Unstitch', 'Barrel', 'Carton', 'peice', 'Sets', 'Unit(s)', 'one',
       'Bag', 'Selfie Kurtis', 'Pound', 'Peice', '1piece', 'No',
       'Kilogram', 'Number', "40' Container", '100 pic', 'Year',
       "20' Container", 'Pcs', 'Piece(s) Onwards', 'One', 'kurtI',
       'Ounce', 'onwards', '10-10000', 'per piese', 'Day', 'Set(s)',
       '12 units', '1pis', 'Onwards', 'One peace', 'Set(S)', 'Pair piece',
       'Ounce(s)', 'Packet(s)', 'Single', 'one pcs', 'Pc', '4 pcs',
       '4 units', 5, 'set', 'kurti', 'pcs', 'pices', 'Pics',
       'single piece', 'pi', 'per piece', 'Kurti', 'one unit', '1 piece',
       '1 pice', 'Suit', 'pair piece', 10, 'pieces', '1pcs', '1 pc',
       '3 set', 'Xl size', '1000 per unit', '1pcd', 'Psc', 'Gram', 'Rs',
       'in', '1 pcs', 'Feet',

In [5]:
len(data['Unit'].unique())

97

In [6]:
data['Unit']=data['Unit'].str.lower()
reg={'pair.*':'pair','pack.*':'pack','set.*':'set','.*piece.*':'piece','dollar.*':'dollar',
     '.*per piece':'unit','meter.*':'meter','1pc':'unit','unstitch':'unit','peice':'piece',
     'no':'number','100 pic':'piece','170 per piece':'piece',"40' container":'container',
     "20' container":'container','onwards':'unit','.*unit':'unit','pices':'piece','pics':'piece',
     'one unit':'unit','.*pcs':'piece','per piese':'piece','1pis':'piece','one peace':'piece',
     '1 pice':'piece','one peace':'piece','.*pc':'piece','onepiece':'piece','1numbers':'number',
     'one':'piece','ounce.*':'ounce','single':'unit','.*set':'set','unit.*':'unit'
    }
data['Unit']=data[['Unit']].replace({'Unit':reg}, regex=True)

In [7]:
len(data['Unit'].unique())

31

In [9]:
'''
Ouput: dictionary 


'''
output={}
##Creating Subset based on Categories
data1=data.groupby('Category Name')
for category,data_df in data1:
    ## loop through each category
    unit_df =data_df.groupby('Unit')
    for unit_type,unit_df in unit_df:
        ##Loop through each unit
        if(len(unit_df)>2):
            #Clustering can be performed only on data set that has no. of records > no. of set cluster                   
            y_means = KMeans(n_clusters=2).fit(unit_df['Price'].values.reshape(-1,1)).labels_
            
            unit_df['predict']=y_means
            #non_anomaly list
            non_anomalies = []
            
            ##selecting the predominant cluster
            frequent_cluster_df=unit_df.groupby(['predict']).filter(lambda x: len(x)==unit_df.groupby(['predict']).size().max())
            #Calculating Standard deviation on the given subset
            random_data_std = np.std(frequent_cluster_df['Price'])
            #Calculating mean on the given subset
            random_data_mean = np.mean(frequent_cluster_df['Price'])
            anomaly_cut_off = random_data_std * 3
            #Lower limit: below 3 standard deviation from mean
            lower_limit  = random_data_mean - anomaly_cut_off 
            #Above 3 standard deviation from mean
            upper_limit = random_data_mean + anomaly_cut_off
            
            for outlier in frequent_cluster_df['Price']:
                if outlier > upper_limit or outlier < lower_limit:
                    pass
                else:
                    non_anomalies.append(outlier)
            #Dictionary with min and max for a given category and unit
            di_temp={"category":category,"min_range":min(non_anomalies),"max_range":max(non_anomalies)}
            temp=category+'_'+unit_type
            output[temp]=di_temp
            
        else:
            #For subset with no. of records less than 2
            di_temp={"category":category,"min_range":min(unit_df['Price']),"max_range":max(unit_df['Price'])}
            temp=category+'_'+unit_type
            output[temp]=di_temp

In [10]:
final_output_data=pd.DataFrame(output).T

In [11]:
final_output_data['Unit']=final_output_data.index

In [12]:
final_output_data

,category,max_range,min_range,Unit
Impact Drill_kit,Impact Drill,5690,2243,Impact Drill_kit
Impact Drill_number,Impact Drill,12360,1250,Impact Drill_number
Impact Drill_pack,Impact Drill,13790,1069,Impact Drill_pack
Impact Drill_piece,Impact Drill,12300,2,Impact Drill_piece
Impact Drill_set,Impact Drill,4895,4895,Impact Drill_set
Impact Drill_unit,Impact Drill,13290,1199,Impact Drill_unit
Ladies Kurta_10-10000,Ladies Kurta,399,399,Ladies Kurta_10-10000
Ladies Kurta_bag,Ladies Kurta,1700,220,Ladies Kurta_bag
Ladies Kurta_barrel,Ladies Kurta,250,180,Ladies Kurta_barrel
Ladies Kurta_box,Ladies Kurta,900,150,Ladies Kurta_box
